# API端口setting

In [ ]:
# This is a sample Python script.
import sys
# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.
# from tensorflow import keras
from flask import Flask, request
import jieba
app = Flask(__name__)

In [ ]:
@app.route('/')
def hello_world():
    return 'Hello World!'

In [ ]:
@app.route('/test',methods=['POST','GET'])
def predictText():
    # 1 接受程序的参数  
    content = request.form.get("content")
    content = str(content)

    result =jieba.cut(content)
    result = '/'.join(result)
    ss = ''
    for r in result:
        ss += r

#    result = "来电|人|反映|：|槐荫区|发祥|一号|公馆|1|楼|（|不清楚|门牌号|）|有人|正在|装修|改造|饭店|，|每天|不到|8|点|就|开始|施工|，|声音|不仅|扰民|，|整个|楼|的|地板|与|窗子|都|在|震动|，|认为|存在|安全|隐患|，|向|彼德|物业|反映|也没有|效果|，|希望|相关|单位|帮助|协调|落实|。|请|处理|"
    return ss

In [ ]:
@app.route('/HotCon',methods=['POST','GET'])
def HotCon():
    # 1 接受程序的参数
    content = request.args.get('content')
    getno = request.args.get('getno')
    starttime = request.args.get('starttime')
    overtime = request.args.get(' overtime')
    sign= request.args.get('sign')
     # id|词|wtflcode|wtflname|createtime|sign   words_ids_wtfls
    # |id|content|wtflcode|wtflname|aftercutcontent|aftercutbystopwords|createtime|sign|

   # select id from  words_ids_wtfls where 词 = cotent and createtime between starttime and overtime and sign =sign
    result = "123|22|2123|123123|123123"
    return result

In [ ]:
if __name__ == '__main__':
   app.run(host='127.0.0.1', port=5000)

# For internship and connect to PostgreSQL

In [ ]:
# This is a sample Python script.
import sys
# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.
# from tensorflow import keras
from flask import Flask, request
import jieba
import psycopg2
import pandas as pd
import jieba.posseg as pseg
import re
app = Flask(__name__)
@app.route('/HotCon',methods=['POST','GET'])
def HotCon():
    conn = psycopg2.connect(database="yw_wordservice", user="postgres",
                        password="123456", host="192.168.50.111", port="5432")
#conn = podbc.connect("Driver={PostgreSQL};Server=,5432;Database=yw_wordservice;uid=postgres;pwd=123456")
    cursor = conn.cursor()
   
#sql="insert n business_info (guid,business_content_cut) VALUES('2','来电|人|反映|：|槐荫区|发祥|一号|公馆|1|楼|')"
    sql = """select business_content_cut from business_info;"""
    s=[]
    try:
        cursor.execute(sql)
        records = cursor.fetchall()
        list = []
        r='[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n。！，]+'
        
        for row in records:
            words =str(row).split('/')
            
            for w in words:
                w=re.sub(r,'',w)
                list.append(w)    

        dict = {}
        for key in list:    
            dict[key] = dict.get(key, 0) + 1
            
        s=sorted(dict.items(),key=lambda item:item[1], reverse=True)
        
        print(s[0])
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)          
            
            
            
    try: 
        p =str(s[0]).split('\'')
        print (p[1])
        sql = """select business_id from business_info where business_content_cut like '%%%s%%'""" %(p[1])
        cursor.execute(sql)
        records2 = cursor.fetchall()
        ss=''+str(s[0])
        for id2 in records2:
            ss+=str(id2)
    
    
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    conn.commit()
    print("ok")

    
    conn.close()


    return ss 
    
if __name__ == '__main__':
   app.run(host='127.0.0.1', port=5000)